#### 1° String de conexão (ajuste conforme seu ambiente)

In [7]:
from sqlalchemy import create_engine, text

server = 'DESKTOP-F0DI39E'
database = 'master'  # criaremos o banco a partir do master
driver = 'ODBC Driver 17 for SQL Server'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

engine = create_engine(connection_string)


#### 2° Criar a base de dados via Python

In [8]:
with engine.connect() as conn:
    conn.execute(text("""
        IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'BI_Projeto')
        CREATE DATABASE BI_Projeto;
    """))
    conn.commit()


ProgrammingError: (pyodbc.ProgrammingError) ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Instrução CREATE DATABASE não permitida em transação com várias instruções. (226) (SQLExecDirectW)')
[SQL: 
        IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'BI_Projeto')
        CREATE DATABASE BI_Projeto;
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)

#### 3° Conectar na base criada

In [9]:
database = 'BI_Projeto'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

engine = create_engine(connection_string)


#### 4° Criação das tabelas

In [10]:
create_table_sql = """
IF NOT EXISTS (
    SELECT * FROM sys.tables 
    WHERE name = 'dbo.Cliente'
)
BEGIN
    CREATE TABLE dbo.Cliente (
        id_Cliente INT IDENTITY(1,1) PRIMARY KEY,
        str_Cliente VARCHAR(150) NOT NULL
    );
END
"""

with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()


#### 5° Inserir dados via Python

In [11]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-F0DI39E;"
    "DATABASE=BI_Projeto;"
    "Trusted_Connection=yes;"
)

cursor = conn.cursor()

# Truncate
cursor.execute("TRUNCATE TABLE dbo.Cliente")
conn.commit()

# Load Excel
df = pd.read_excel("base_projetos.xlsx", sheet_name="dbo.Cliente")

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO dbo.Cliente (str_Cliente)
        VALUES (?)
    """, row['str_Cliente'])

conn.commit()
cursor.close()
conn.close()


#### (Opcional) Consultar os dados

In [13]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dbo.Cliente"))
    for row in result:
        print(row)
